In [2]:
import numpy as np
import matplotlib.pyplot as plt

from create_prepared_data_tsfresh import *
from compression import calculateCompRatio, compress_dataset
from utils.personal_utils import *

import os
import pywt

import matplotlib.pyplot as plt
import time

In [2]:
all_ds = {
    'AppliancesEnergy':   '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts',
    'NewsTitleSentiment': '/home/sim/Desktop/TS Extrinsic Regression/data/NewsTitleSentiment_TEST.ts',
    'BenzeneConcentration':'/home/sim/Desktop/TS Extrinsic Regression/data/BenzeneConcentration_TEST.ts',
    'BeijingPM25Quality': '/home/sim/Desktop/TS Extrinsic Regression/data/BeijingPM25Quality_TEST.ts',

    'IEEEPPG':            '/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TEST.ts',
    'FloodModeling1':     '/home/sim/Desktop/TS Extrinsic Regression/data/FloodModeling1_TEST.ts',
    'HouseholdPowerConsumption1': '/home/sim/Desktop/TS Extrinsic Regression/data/HouseholdPowerConsumption1_TEST.ts',
    'Covid3Month':              '/home/sim/Desktop/TS Extrinsic Regression/data/Covid3Month_TEST.ts'
}

Steps: Change common features to the common features of each starting test set!
All other things stay the same!

FOUND OUT: For each Dataset and Technique, the number of features of tsfresh are the same! Also for each Dataset we get the same features after tsfresh for each tq and compression strength

In [7]:
# Code to show, that Each Datast has the same features
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97, 0.99]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month'
]

tqs = ['dct', 'dft', 'dwt']

common_features_dict = {}

# Helper function to find common elements in lists
def common_elements(lists):
    if not lists:
        return []
    common = lists[0]
    for lst in lists[1:]:
        common = [elem for elem in common if elem in lst]
    return common

# Iterate over them to go for each value
for ds_name in ds_names:


    for tq in tqs:
        path_to_runs = log_path + f'{ds_name}_Runs/'
        common_features = None
        features_lists = []

        
        for dval in dropout_values:
            if dval is None:
                 curr_df = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/NONE_NONE_{ds_name}_features_TEST')
                 features_lists.append(curr_df.columns)
                 print(len(curr_df.columns))

            else:
                curr_df = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/{dval}_{tq}_{ds_name}_features_TEST')
                features_lists.append(curr_df.columns)
                print(len(curr_df.columns))
            
        
        common_features = common_elements(features_lists)
        common_features_dict[(ds_name, tq)] = common_features





# Print overlapping for (Dataset,Comp_Alg) combinaton
for (ds_name, tq), common_features in common_features_dict.items():
    if common_features is not None:
        print(f"Number of overlapping features for {ds_name}: {tq} {len(common_features)}")
print()



# Print num of overlapping for each tq
overlap_per_tq = {}
for tmp_tq in tqs:
        overlap_per_tq[tmp_tq] = common_elements([common_features for (ds_name, tq), common_features in common_features_dict.items() if tq == tmp_tq])
        print(f"Number of overlapping features for whole {tmp_tq} {len(overlap_per_tq[tmp_tq])}")
print()


# Print number of overlapping for each dataset
overlapp_per_dataset = {}
for tmp_dsname in ds_names:

        overlapp_per_dataset[tmp_dsname] = common_elements([common_features for (ds_name, tq), common_features in common_features_dict.items() if ds_name == tmp_dsname])
        print(f"Number of overlapping features for whole {tmp_dsname} {len(overlapp_per_dataset[tmp_dsname])}")

18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
18649
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6217
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
6994
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
3886
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
778
Number of overlapping features for AppliancesEnergy: dct 18649
Number of overlapping features for AppliancesEnergy: dft 18649
Number of overlapping features for AppliancesEnergy: dwt 1

In [10]:
# Calculate the correlation, we get a dict looking like: correlations[('AppliancesEnergy', 'dft', 0.5)] -> correlation
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97, 0.99]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    #'NewsTitleSentiment',
    #'BenzeneConcentration',
    #'BeijingPM25Quality',
    #'IEEEPPG',
    #'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month',
    #'AppliancesEnergy',
]


tqs = ['dct', 'dft', 'dwt']

prepared_dfs = {}
correlations = {}

for ds_name in ds_names:
    path_to_runs = log_path + f'{ds_name}_Runs/'



    features_raw = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/NONE_NONE_{ds_name}_features_TEST')
    #features_raw = load_and_prepare_everything(all_ds[ds_name], None, None)[1].drop('target', axis=1)
    
    path_to_pred = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/predictions.npy'
    prediction_raw = np.load(path_to_pred)
    features_raw['prediction'] = prediction_raw


    for tq in tqs:
        prepared_dfs[(ds_name,tq,None)] = features_raw.copy() # bc different features for each technique

        for dval in dropout_values[1:]:
            

            features = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/{dval}_{tq}_{ds_name}_features_TEST')
            #features = load_and_prepare_everything(all_ds[ds_name], tq, dval)[1].drop('target', axis=1)
            
            path_to_pred = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/predictions.npy'

            with open(path_to_pred, 'r') as file:
                prediction = np.load(path_to_pred)


            features['prediction'] = prediction
            prepared_dfs[(ds_name,tq, dval)] = features.copy()


            # Add Subtraction and Correlation
            sub_df = (features_raw - features)
            corrs = sub_df.corrwith(sub_df['prediction'])
            correlations[(ds_name,tq,dval)] = corrs

            print(f'{ds_name}, {tq}, {dval} correlation calculated')
            


HouseholdPowerConsumption1, dct, 0.5 correlation calculated
HouseholdPowerConsumption1, dct, 0.75 correlation calculated
HouseholdPowerConsumption1, dct, 0.85 correlation calculated
HouseholdPowerConsumption1, dct, 0.9 correlation calculated
HouseholdPowerConsumption1, dct, 0.95 correlation calculated
HouseholdPowerConsumption1, dct, 0.97 correlation calculated
HouseholdPowerConsumption1, dct, 0.99 correlation calculated
HouseholdPowerConsumption1, dft, 0.5 correlation calculated
HouseholdPowerConsumption1, dft, 0.75 correlation calculated
HouseholdPowerConsumption1, dft, 0.85 correlation calculated
HouseholdPowerConsumption1, dft, 0.9 correlation calculated
HouseholdPowerConsumption1, dft, 0.95 correlation calculated
HouseholdPowerConsumption1, dft, 0.97 correlation calculated
HouseholdPowerConsumption1, dft, 0.99 correlation calculated
HouseholdPowerConsumption1, dwt, 0.5 correlation calculated
HouseholdPowerConsumption1, dwt, 0.75 correlation calculated
HouseholdPowerConsumption1, d

In [11]:
# Get Average Correlation of absolute value over each (ds_name, tq)
# The Entries are Series object, the row name is the featuer name!
corr_ds_name_tq = {}


for ds_name in ds_names:
    

    for tq in tqs:

        # Initalize a df with same columns, rows but zero in each field
        avg_corr = correlations[(ds_name,tq,0.5)].copy()
        avg_corr[:] = 0
        #print(avg_corr)

        for d_val in dropout_values[1:]:
            curr_corr_df = correlations[(ds_name,tq,d_val)].abs()  # take abs value
            avg_corr = curr_corr_df + avg_corr
        
        avg_corr = avg_corr / len(dropout_values[1:])
    
        
        corr_ds_name_tq[(ds_name, tq)] = avg_corr.sort_values(ascending=False)



corr_ds_name = {}
for ds_name in ds_names:
    corr_ds_name[ds_name] = corr_ds_name_tq[(ds_name, 'dct')]
    corr_ds_name[ds_name] += corr_ds_name_tq[(ds_name, 'dft')]
    corr_ds_name[ds_name] += corr_ds_name_tq[(ds_name, 'dwt')]
    
    corr_ds_name[ds_name] = corr_ds_name[ds_name].sort_values(ascending=False) / 3
    




In [5]:
# Get Correlation over each dataset

for ds_name in ds_names:
    print(ds_name )
    print(corr_ds_name[(ds_name)])
    print()


HouseholdPowerConsumption1
prediction                                                        1.000000
dim_5__autocorrelation__lag_3                                     0.187021
dim_5__change_quantiles__f_agg_var__isabs_True__qh_1.0__ql_0.0    0.186411
dim_5__autocorrelation__lag_2                                     0.184919
dim_5__abs_energy                                                 0.184884
                                                                    ...   
dim_5__symmetry_looking__r_0.9500000000000001                          NaN
dim_5__value_count__value_-1                                           NaN
dim_5__value_count__value_0                                            NaN
dim_5__value_count__value_1                                            NaN
target                                                                 NaN
Length: 3887, dtype: float64

Covid3Month
prediction                                                        1.000000
dim_1__fft_coefficient__attr_an

In [6]:
# Print for Each Combination the top 5 features
for ds_name in ds_names:
    for tq in tqs:
        print(ds_name + " " + tq)
        print(corr_ds_name_tq[(ds_name, tq)].head(6))
        print()



HouseholdPowerConsumption1 dct
prediction                               3.000000
dim_5__autocorrelation__lag_2            0.554756
dim_5__cid_ce__normalize_False           0.551622
dim_5__partial_autocorrelation__lag_1    0.545708
dim_5__autocorrelation__lag_1            0.545708
dim_5__cid_ce__normalize_True            0.547739
dtype: float64

HouseholdPowerConsumption1 dft
prediction                        1.000000
dim_5__abs_energy                 0.232301
dim_5__variance                   0.232301
dim_5__autocorrelation__lag_3     0.231683
dim_5__autocorrelation__lag_2     0.230032
dim_5__cid_ce__normalize_False    0.230000
dtype: float64

HouseholdPowerConsumption1 dwt
prediction                                                       1.000000
dim_2__ratio_value_number_to_time_series_length                  0.238860
dim_2__sum_of_reoccurring_values                                 0.203753
dim_2__percentage_of_reoccurring_datapoints_to_all_datapoints    0.190692
dim_2__lempel_ziv_com

In [16]:
# To sort features

import re
def extract_fifth_number(column_name):
    match = re.match(r'^.{4}(\d+)', column_name)  # Match numbers after the first 4 characters
    if match:
        return int(match.group(1))
    return float('inf')  # Use infinity for columns without a valid number

# Create a dictionary with column names as keys and extracted numbers as values
column_sort_order = {col: extract_fifth_number(col) for col in features.columns}

# Sort columns based on the extracted numbers
sorted_columns = sorted(column_sort_order, key=column_sort_order.get)

# Reorder the DataFrame columns
features_sorted = features[sorted_columns]
features_sorted

,dim_1__variance_larger_than_standard_deviation,dim_1__has_duplicate_max,dim_1__has_duplicate_min,dim_1__has_duplicate,dim_1__sum_values,dim_1__abs_energy,dim_1__mean_abs_change,dim_1__mean_change,dim_1__mean_second_derivative_central,dim_1__median,...,dim_1__fourier_entropy__bins_100,dim_1__permutation_entropy__dimension_3__tau_1,dim_1__permutation_entropy__dimension_4__tau_1,dim_1__permutation_entropy__dimension_5__tau_1,dim_1__permutation_entropy__dimension_6__tau_1,dim_1__permutation_entropy__dimension_7__tau_1,dim_1__query_similarity_count__query_None__threshold_0.0,dim_1__mean_n_absolute_max__number_of_maxima_7,target,prediction
0,0.0,0.0,0.0,1.0,0.000000,25.000000,0.170387,0.000000,0.0,0.000000,...,3.696721,0.131571,0.199100,0.26783,0.337792,0.341279,0.0,1.010153,0.011884,0.035518
1,0.0,1.0,1.0,1.0,10.000000,12.500000,0.015060,0.015060,0.0,0.000000,...,0.329684,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,1.250000,0.003795,0.033880
2,0.0,1.0,1.0,1.0,-2.928932,25.000000,0.036359,0.036359,0.0,0.000000,...,1.432289,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,1.545867,0.082988,0.036872
3,0.0,1.0,1.0,1.0,7.071068,25.000000,0.031948,0.031948,0.0,0.000000,...,1.259314,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,1.388960,0.045109,0.052389
4,0.0,1.0,1.0,1.0,-5.656854,30.500000,0.036207,0.036207,0.0,0.000000,...,1.397956,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,1.590990,0.127831,0.036872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.0,1.0,1.0,1.0,7.071068,12.500000,0.021298,0.021298,0.0,0.000000,...,0.220352,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,1.010153,0.021028,0.036872
57,0.0,1.0,1.0,1.0,-7.071068,12.500000,0.021298,0.021298,0.0,0.000000,...,0.220352,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,1.010153,0.010999,0.044085
58,0.0,0.0,0.0,1.0,0.000000,49.000000,0.238542,0.000000,0.0,0.000000,...,3.664482,0.262533,0.265153,0.26783,0.270567,0.273365,0.0,1.414214,0.181818,0.035518
59,0.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,1.397956,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.0,0.000000,0.000000,0.035518


In [17]:
file_path = '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

selected_features = features[lines]


column_sort_order = {col: extract_fifth_number(col) for col in selected_features.columns}

# Sort columns based on the extracted numbers
sorted_columns = sorted(column_sort_order, key=column_sort_order.get)

# Reorder the DataFrame columns
sfeatures_sorted = selected_features[sorted_columns]
sfeatures_sorted

FileNotFoundError: [Errno 2] No such file or directory: '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'